In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
czi_dfs = pd.read_excel('./czi_software_citation_intent_data.xlsx', sheet_name=["comm_sample_200", "non_comm_sample_200"])
merged_dfs = pd.read_excel('./software_intent_data_merged.xlsx', sheet_name=["final_data_w_context", "final_data", "unlabeled"])

In [ ]:
# assign numerical labels
num2label = {0: "created", 1: "used", 2: "mention", 3: "none"}
label2num = {"created": 0, "used": 1, "mention": 2, "none": 3}

In [ ]:
# remove excel column parsing errors
for df_dict in [czi_dfs, merged_dfs]:
    for df in df_dict.values():
        unnamed_cols = [c for c in df.columns if c.startswith("Unnamed")]
        if len(unnamed_cols) > 0:
            df.drop(columns=unnamed_cols, inplace=True)
merged_df = merged_dfs["final_data"].drop(columns=["context"])
# merge context data in
merged_df = pd.merge(merged_df, merged_dfs["final_data_w_context"], on=["id", "sentence", "used", "created", "mention"], how="left")
# concatenate unlabelled data
merged_df = pd.concat([merged_df, merged_dfs["unlabeled"]])
czi_df = pd.concat(list(czi_dfs.values()))
czi_df = czi_df.dropna(subset="Intent (creation, used, mention)")
merged_df["label"] = np.where(
    merged_df["created"], label2num["created"], np.where(
        merged_df["used"], label2num["used"], np.where(
            merged_df["mention"], label2num["mention"], label2num["none"]
        )
    )
)
merged_df["text"] = merged_df["sentence"]
merged_df = merged_df.drop_duplicates(subset="text")
czi_df["label"] = np.where(
    czi_df["Intent (creation, used, mention)"] == "creation", label2num["created"], np.where(
        czi_df["Intent (creation, used, mention)"] == "used", label2num["used"], np.where(
            czi_df["Intent (creation, used, mention)"] == "mention", label2num["mention"], label2num["none"]
        )
    )
)

In [ ]:
merged_df.to_csv("./software_citation_intent_merged.csv")
czi_df.to_csv("./software_citation_intent_czi.csv")